In [4]:
!pip install gdown
!pip install --upgrade datasets
!pip install tokenizers
!pip install transformers==4.46.1
!pip install torch==2.5.1
!pip install jiwer
!pip install evaluate

  Using cached torch-2.5.1-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-p

In [2]:
!gdown https://drive.google.com/uc?id=1DFbOzcWTEAk3nt7HJtkzFdHqq5MRt7o-

Downloading...
From (original): https://drive.google.com/uc?id=1DFbOzcWTEAk3nt7HJtkzFdHqq5MRt7o-
From (redirected): https://drive.google.com/uc?id=1DFbOzcWTEAk3nt7HJtkzFdHqq5MRt7o-&confirm=t&uuid=5840a187-e624-4e10-ac14-be7b82b4be10
To: /content/Doctor+Own.zip
100% 82.1M/82.1M [00:02<00:00, 39.5MB/s]


In [3]:
!unzip /content/Doctor+Own.zip

Streaming output truncated to the last 5000 lines.
  inflating: Doctor+Own/Images/2435.png  
  inflating: Doctor+Own/Images/2436.png  
  inflating: Doctor+Own/Images/2437.png  
  inflating: Doctor+Own/Images/2438.png  
  inflating: Doctor+Own/Images/2439.png  
  inflating: Doctor+Own/Images/244.jpg  
  inflating: Doctor+Own/Images/244.png  
  inflating: Doctor+Own/Images/2440.png  
  inflating: Doctor+Own/Images/2441.png  
  inflating: Doctor+Own/Images/2442.png  
  inflating: Doctor+Own/Images/2443.png  
  inflating: Doctor+Own/Images/2444.png  
  inflating: Doctor+Own/Images/2445.png  
  inflating: Doctor+Own/Images/2446.png  
  inflating: Doctor+Own/Images/2447.png  
  inflating: Doctor+Own/Images/2448.png  
  inflating: Doctor+Own/Images/2449.png  
  inflating: Doctor+Own/Images/245.jpg  
  inflating: Doctor+Own/Images/245.png  
  inflating: Doctor+Own/Images/2450.png  
  inflating: Doctor+Own/Images/2451.png  
  inflating: Doctor+Own/Images/2452.png  
  inflating: Doctor+Own/Image

In [1]:
import pandas as pd

df = pd.read_csv('/content/Doctor+Own/Final_metadata.txt',
                 sep='\t',  # Specify tab as the delimiter
                 header=None,  # No header in the file
                 names=["file_name", "text"])  # Set column names

# Fix file names that end with "jp" instead of "jpg"
df['file_name'] = df['file_name'].apply(lambda x: x + 'g' if x.endswith('jp') else x)

# Display the first few rows to verify
print(df.head())
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2)
# we reset the indices to start from zero
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
import torch
from torch.utils.data import Dataset
from PIL import Image

class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text
        file_name = self.df['file_name'][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text,
                                          padding="max_length",
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
train_dataset = IAMDataset(root_dir='/content/Doctor+Own/Images/',
                           df=train_df,
                           processor=processor)
eval_dataset = IAMDataset(root_dir='/content/Doctor+Own/Images/',
                           df=test_df,
                           processor=processor)

# Print dataset sizes
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

# Check encoding for the first training example
encoding = train_dataset[0]
for k, v in encoding.items():
    print(k, v.shape)


  file_name                                 text
0     1.jpg  Augmentin DDS Suspension 5ml 2times
1     2.jpg                Almox 500mg Tab1 8hrs
2     3.jpg   Ascoril LS Junior Syrup 5ml 2times
3     4.jpg            Aptimust Syrup 5ml 2times
4     5.jpg                Alfoo 10mg Tab1 daily


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Number of training examples: 2701
Number of validation examples: 676
pixel_values torch.Size([3, 384, 384])
labels torch.Size([128])


In [17]:
!pip install torch==2.4.0 torchvision==0.19.0

  Using cached torchvision-0.19.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
# Get the file name and construct the full path
file_name = train_df['file_name'][0]
file_path = os.path.join(train_dataset.root_dir, file_name)

# Print the full path for debugging
print(f"Attempting to open: {file_path}")

# Check if the file exists before opening
if os.path.exists(file_path):
    image = Image.open(file_path).convert("RGB")
else:
    print(f"File not found: {file_path}")


Attempting to open: /content/Doctor+Own/Images/1654.png


In [3]:
labels = encoding['labels']
labels[labels == -100] = processor.tokenizer.pad_token_id
label_str = processor.decode(labels, skip_special_tokens=True)
print(label_str)

Ketozol


In [4]:
from transformers import VisionEncoderDecoderModel

model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")

config.json:   0%|          | 0.00/4.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": false,
  "transformers_version": "4.46.1"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [6]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    fp16=True,
    output_dir="./model",
    logging_steps=2,
    save_steps=1000,
    eval_steps=200,
    num_train_epochs=5
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
from evaluate import load

# Load the CER metric from Hugging Face datasets
cer_metric = load("cer")

In [9]:
!pip install jellyfish


In [12]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [13]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)

trainer.train()

<ipython-input-13-c8db78083648>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss,Validation Loss,Cer
200,0.711300,0.859861,0.174442
400,0.787700,0.723072,0.157895
600,0.391300,0.605662,0.103668
800,0.501300,0.575129,0.091507
1000,0.429100,0.532475,0.077552
1200,0.344300,0.497589,0.064195
1400,0.298200,0.451605,0.050439
1600,0.344900,0.435226,0.046850


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=1690, training_loss=0.540333333117722, metrics={'train_runtime': 1973.3929, 'train_samples_per_second': 6.844, 'train_steps_per_second': 0.856, 'total_flos': 1.1950513061174968e+19, 'train_loss': 0.540333333117722, 'epoch': 5.0})

In [22]:
import os
import torch
import jellyfish
import random
import csv
from PIL import Image

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to read ground truth from the text file
def load_ground_truths(text_file):
    ground_truths = {}
    with open(text_file, "r", encoding="utf-8") as file:
        for line in file:
            parts = line.strip().split("\t")  # Split by tab
            if len(parts) == 2:
                image_name, text = parts
                ground_truths[image_name] = text  # Store in dictionary
    return ground_truths

# Function to calculate Jaro and Levenshtein similarity
def calculate_similarity_metrics(predicted_text, ground_truth):
    jaro_sim = jellyfish.jaro_similarity(predicted_text, ground_truth)
    lev_dist = jellyfish.levenshtein_distance(predicted_text, ground_truth)
    max_len = max(len(predicted_text), len(ground_truth))
    lev_ratio = 1 - (lev_dist / max_len) if max_len > 0 else 1  # Normalize to [0,1]
    return jaro_sim, lev_ratio

# Function to predict text for a single image
def predict_image(image_path, ground_truth):
    image = Image.open(image_path).convert("RGB")
    pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)

    with torch.no_grad():
        generated_ids = model.generate(pixel_values)

    predicted_text = processor.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Compute similarity metrics
    jaro_sim, lev_ratio = calculate_similarity_metrics(predicted_text, ground_truth)

    return predicted_text, jaro_sim, lev_ratio

# Function to process multiple images from a folder
def process_images(folder_path, ground_truths, num_images=3000, output_csv="results.csv"):
    all_images = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    selected_images = random.sample(all_images, min(num_images, len(all_images)))  # Select 400 images randomly

    results = []
    total_jaro, total_lev_ratio = 0, 0
    count = 0

    for image_name in selected_images:
        image_path = os.path.join(folder_path, image_name)
        ground_truth = ground_truths.get(image_name, "")  # Get ground truth text

        predicted_text, jaro_sim, lev_ratio = predict_image(image_path, ground_truth)
        results.append((image_name, predicted_text, ground_truth, jaro_sim, lev_ratio))

        total_jaro += jaro_sim
        total_lev_ratio += lev_ratio
        count += 1

    # Compute averages
    avg_jaro = total_jaro / count if count > 0 else 0
    avg_lev_ratio = total_lev_ratio / count if count > 0 else 0

    # Save results to CSV
    with open(output_csv, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Image Name", "Predicted Text", "Ground Truth", "Jaro Similarity", "Levenshtein Ratio"])
        writer.writerows(results)
        writer.writerow([])  # Empty row
        writer.writerow(["Average", "", "", avg_jaro, avg_lev_ratio])  # Save averages

    return results, avg_jaro, avg_lev_ratio

# Example usage
text_file = "/content/Doctor+Own/Final_metadata.txt"  # Path to text file
folder_path = "/content/Doctor+Own/Images/"  # Folder containing images

ground_truths = load_ground_truths(text_file)  # Load ground truths

# Process 400 images
results, avg_jaro, avg_lev_ratio = process_images(folder_path, ground_truths, num_images=400)

# Print the results (printing only first 10 for readability)
for img_name, pred_text, gt_text, jaro, lev in results[:10]:
    print(f"Image: {img_name}")
    print(f"Predicted Text: {pred_text}")
    print(f"Ground Truth: {gt_text}")
    print(f"Jaro Similarity: {jaro:.4f}")
    print(f"Levenshtein Ratio: {lev:.4f}")
    print("-" * 50)

# Print the averages
print(f"Average Jaro Similarity: {avg_jaro:.4f}")
print(f"Average Levenshtein Ratio: {avg_lev_ratio:.4f}")

print("Results saved to results.csv")


Image: 854.png
Predicted Text: Denixil
Ground Truth: Denixil
Jaro Similarity: 1.0000
Levenshtein Ratio: 1.0000
--------------------------------------------------
Image: 2519.png
Predicted Text: Opton
Ground Truth: Opton
Jaro Similarity: 1.0000
Levenshtein Ratio: 1.0000
--------------------------------------------------
Image: 1209.png
Predicted Text: Fexofast
Ground Truth: Fexofast
Jaro Similarity: 1.0000
Levenshtein Ratio: 1.0000
--------------------------------------------------
Image: 2218.png
Predicted Text: Napa
Ground Truth: Napa
Jaro Similarity: 1.0000
Levenshtein Ratio: 1.0000
--------------------------------------------------
Image: 155.jpg
Predicted Text: Adoxa 150mg 2times daily
Ground Truth: Adoxa 150mg 2times daily
Jaro Similarity: 1.0000
Levenshtein Ratio: 1.0000
--------------------------------------------------
Image: 1849.png
Predicted Text: Maxima
Ground Truth: Maxima
Jaro Similarity: 1.0000
Levenshtein Ratio: 1.0000
--------------------------------------------------
